In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- 경로 설정 ---
ROOT_DIR = Path.cwd().parent 
DATA_DIR = ROOT_DIR / "BIgcontest_Data"
CROLLING_DIR = ROOT_DIR / "Crolling"
EDA_DIR = ROOT_DIR / "EDA"
MODEL_DIR = ROOT_DIR / "Model"

# ==============================================================================
# 1. 필요 데이터 불러오기
# ==============================================================================
try:
    # 피처 엔지니어링이 완료된 월별 시계열 데이터
    df_featured = pd.read_csv(EDA_DIR/"featured_data.csv")
    # VGAE 모델이 생성한 가게별 잠재 벡터
    df_embeddings = pd.read_csv(MODEL_DIR/"node_embeddings.csv")
    print("✅ Step 1: 데이터 로딩 성공!")
except FileNotFoundError:
    print("❌ 오류: 필요한 CSV 파일을 찾을 수 없습니다. 이전 단계들을 확인해주세요.")
    exit()

✅ Step 1: 데이터 로딩 성공!


In [2]:
# ==============================================================================
# 2. 데이터 병합 및 정렬
# ==============================================================================
# 월별 데이터에 가게별 잠재 벡터(고정값)를 추가
df_rl = pd.merge(df_featured, df_embeddings, on='ENCODED_MCT')

# 각 가게(ENCODED_MCT)별로 시간을 오름차순으로 정렬
df_rl = df_rl.sort_values(by=['ENCODED_MCT', 'TA_YM']).reset_index(drop=True)

print("✅ Step 2: 데이터 병합 및 정렬 완료!")

✅ Step 2: 데이터 병합 및 정렬 완료!


In [3]:
# ==============================================================================
# 3. 강화학습 요소(상태, 행동, 보상) 정의 및 생성
# ==============================================================================
# --- 3-1. 상태 (State) 정의 ---
# 상태는 '가게의 잠재 벡터'와 '월별 실적 수치'의 조합으로 정의
latent_cols = [f'latent_{i}' for i in range(16)]
monthly_feature_cols = [
    'DLV_SAA_RAT',         # 배달 매출 비중
    'MCT_UE_CLN_REU_RAT',  # 재방문 고객 비율
    'MCT_UE_CLN_NEW_RAT',  # 신규 고객 비율
    'RC_M1_SAA_RANK',      # 매출 순위
    'TREND_RATIO'          # 업종 트렌드 지수
]
state_cols = latent_cols + monthly_feature_cols


# --- 3-2. 행동 (Action) 정의 ---
# 실제 행동 데이터가 없으므로, 월별 지표의 '변화량'을 행동으로 간주(Proxy)합니다.
# 예: "배달 비중이 10% 늘었다" -> "배달 강화" 행동으로 해석
# 여기서는 간단하게 4가지 조합으로 행동을 이산화(Discretize)합니다.
# (배달 비중 변화, 신규 고객 비중 변화) -> (증가/감소, 증가/감소)
# 0: (감소, 감소), 1: (감소, 증가), 2: (증가, 감소), 3: (증가, 증가)

# '다음 달'의 데이터를 가져와 변화량을 계산하기 위해 shift(-1) 사용
df_rl['DLV_RAT_CHANGE'] = df_rl.groupby('ENCODED_MCT')['DLV_SAA_RAT'].shift(-1) - df_rl['DLV_SAA_RAT']
df_rl['NEW_RAT_CHANGE'] = df_rl.groupby('ENCODED_MCT')['MCT_UE_CLN_NEW_RAT'].shift(-1) - df_rl['MCT_UE_CLN_NEW_RAT']

# 변화량의 부호에 따라 행동 정의
actions = []
for _, row in df_rl.iterrows():
    action = 0
    if row['DLV_RAT_CHANGE'] > 0: action += 2
    if row['NEW_RAT_CHANGE'] > 0: action += 1
    actions.append(action)
df_rl['ACTION'] = actions


# --- 3-3. 보상 (Reward) 정의 ---
# 보상은 '매출 순위의 향상 정도'로 정의. 순위가 낮아질수록(좋아질수록) 높은 보상.
# (현재 순위 - 다음 달 순위)
df_rl['REWARD'] = df_rl.groupby('ENCODED_MCT')['RC_M1_SAA_RANK'].shift(-1).rsub(df_rl['RC_M1_SAA_RANK'])


# --- 3-4. 다음 상태 (Next State) 정의 ---
# 다음 달의 상태(State) 데이터를 가져옵니다.
df_rl['NEXT_STATE'] = df_rl.groupby('ENCODED_MCT')[state_cols].shift(-1).apply(lambda x: x.values.tolist(), axis=1)

print("✅ Step 3: 상태, 행동, 보상 정의 및 생성 완료!")

✅ Step 3: 상태, 행동, 보상 정의 및 생성 완료!


In [4]:
# ==============================================================================
# 4. 최종 데이터셋 생성 및 저장
# ==============================================================================
# (상태, 행동, 보상, 다음 상태) 튜플을 만들기 위해 마지막 달 데이터 등 불필요한 행 제거
df_rl_final = df_rl.dropna(subset=['REWARD', 'NEXT_STATE']).reset_index(drop=True)

# 최종 데이터셋 구성
rl_dataset = pd.DataFrame({
    'STATE': df_rl_final[state_cols].values.tolist(),
    'ACTION': df_rl_final['ACTION'],
    'REWARD': df_rl_final['REWARD'],
    'NEXT_STATE': df_rl_final['NEXT_STATE']
})

print("\n--- Step 4: 최종 강화학습 데이터셋 생성 완료! ---")
print(f"  - 총 {len(rl_dataset)}개의 학습용 데이터(Transition) 생성")
print(rl_dataset.head())


# 다음 단계인 오프라인 강화학습 모델(CQL) 학습에 사용할 수 있도록 저장
output_path = "./rl_dataset.csv"
rl_dataset.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n🎉 모든 작업 완료! 강화학습용 최종 데이터가 '{output_path}' 파일로 저장되었습니다.")


--- Step 4: 최종 강화학습 데이터셋 생성 완료! ---
  - 총 72313개의 학습용 데이터(Transition) 생성
                                               STATE  ACTION  REWARD  \
0  [0.23911169, -0.16731161, -0.20430204, -0.3184...       0     0.0   
1  [0.23911169, -0.16731161, -0.20430204, -0.3184...       0     0.0   
2  [0.23911169, -0.16731161, -0.20430204, -0.3184...       1     0.0   
3  [0.23911169, -0.16731161, -0.20430204, -0.3184...       0     0.0   
4  [0.23911169, -0.16731161, -0.20430204, -0.3184...       0     0.0   

                                          NEXT_STATE  
0  [0.23911169, -0.16731161, -0.20430204, -0.3184...  
1  [0.23911169, -0.16731161, -0.20430204, -0.3184...  
2  [0.23911169, -0.16731161, -0.20430204, -0.3184...  
3  [0.23911169, -0.16731161, -0.20430204, -0.3184...  
4  [0.23911169, -0.16731161, -0.20430204, -0.3184...  

🎉 모든 작업 완료! 강화학습용 최종 데이터가 './rl_dataset.csv' 파일로 저장되었습니다.
